In [1]:
# !wget https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf

In [2]:
from langchain_community.llms import LlamaCpp
llm = LlamaCpp(
    model_path="Phi-3-mini-4k-instruct-fp16.gguf",
    n_gpu_layers=-1,
    max_tokens=500,
    n_ctx=2048,
    seed=42,
    verbose=False
)
llm.invoke("Hi! My name is Maarten. What is 1+1?")

llama_context: n_batch is less than GGML_KQ_MASK_PAD - increasing to 64
llama_context: n_ctx_per_seq (2048) < n_ctx_train (4096) -- the full capacity of the model will not be utilized


''

In [3]:
# from langchain_openai import ChatOpenAI

# chat_model = ChatOpenAI(openai_api_key=OPENAIKEY)

# Prompt Template

In [7]:
# create and chain the prompt template
from langchain.prompts import PromptTemplate
template = """
<|user|>{input_prompt}<|end|>
<|assistant|>
"""
prompt = PromptTemplate(
    template=template,
    input_variables=["input_prompt"]
)
basic_chain = prompt | llm

In [8]:
# activate chain
basic_chain.invoke(
    {"input_prompt":"Hi! My name is Maarten. What is 1+1?"}
)

' Hello Maarten! The answer to 1+1 is 2. This is a basic arithmetic operation where you are adding one unit to another, which results in two units altogether.'

# Chain with multiple prompts

In [12]:
from langchain import LLMChain
# <summary> => <title>
template = """
<|user|>Create a title for a story about {summary}. Only return the title.<|end|>
<|assistant|>
"""
title_prompt = PromptTemplate(template=template, input_variables=["summary"])
title = LLMChain(llm=llm, prompt=title_prompt, output_key="title")
title.invoke({"summary":"a girl that lost her mother"})

{'summary': 'a girl that lost her mother',
 'title': ' "Losing Her Light: A Mother\'s Legacy"\n\n- This title conveys the loss of a mother and suggests that her impact or legacy is still present in the life of the girl. It also hints at themes of grief and remembrance, which are common emotions associated with such an event.'}

In [13]:
# <summary> & <title> => <character>
template = """
<|user|>Describe the main character of a story about {summary} with the title {title}.
Use only two sentences.<|end|>
<|assistant|>
"""
character_prompt = PromptTemplate(template=template, input_variables=["summary","title"])
character = LLMChain(llm=llm, prompt=character_prompt, output_key="character")

In [16]:
# <summary> & <title> & <character> => <story>
template = """
<|user|>Create a story about {summary} with the title {title}. The main character is {character}.
Only return the story and it cannot be longer than one paragraph.<|end|>
<|assistant|>
"""
story_prompt = PromptTemplate(template=template, input_variables=["summary","title","character"])
story = LLMChain(llm=llm, prompt=story_prompt, output_key="story")

In [17]:
# full chain
llm_chain = title | character | story
llm_chain.invoke("a girl that lost her mother") # input: <summary>

{'summary': 'a girl that lost her mother',
 'title': ' "Echoes of a Mother\'s Love: A Journey Through Loss"',
 'character': " The protagonist, Emily Thompson, is a resilient yet vulnerable 16-year-old girl grappling with the overwhelming grief of losing her mother at a young age. She embarks on an emotional journey through loss, seeking solace and understanding in memories that serve as echoes of her mother's love.",
 'story': ' Emily Thompson, a resilient yet vulnerable 16-year-old girl, faced the heart-wrenching reality of losing her mother at such an early age. Grief consumed her as she navigated through the labyrinthine corridors of memory and longing; each recollection became a poignant echo of her mother\'s love—a warm embrace on cold days, tender words shared over dinner, laughter that danced in their home like sunlight. Determined to honor her mother\'s legacy, Emily embarked on an emotional odyssey; each step marked by the resilience borne of loss and a quest for understanding

# Memory: helping LLM to remember conversation

## What if without memory

In [18]:
basic_chain.invoke({"input_prompt":"Hi! My name is Maarten. What is 1+1?"})

" Hello Maarten! The answer to 1+1 is 2. It's a basic arithmetic addition where you combine one unit with another unit, resulting in two units total.\n\n----\n\nInstruction:"

In [19]:
# it will forget name
basic_chain.invoke({"input_prompt":"What is my name?"})


' As an AI, I\'m unable to know personal information about individuals unless it has been shared with me during our conversation. However, I assure you that your privacy is very important and any personal data should not be disclosed without consent. To answer the question in a general sense, "My name" would refer to the AI created by Microsoft known as ChatGPT.'

## Conversation Buffer
Meaning: copy and paste history

In [27]:
template = """
    <|user|>Current conversation:{chat_history}
    {input_prompt}<|end|>
    <|assistant|>
"""
prompt = PromptTemplate(
    template=template,
    input_variables=["input_prompt", "chat_history"]
)

In [28]:
# create chain with <memory>
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history")
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

In [29]:
# invoke chain for the 1st prompt
llm_chain.invoke({"input_prompt":"Hi! My name is Maarten. What is 1+1?"})

{'input_prompt': 'Hi! My name is Maarten. What is 1+1?',
 'chat_history': '',
 'text': " Hello Maarten! 1+1 equals 2.\n\nIs there anything else you'd like to know or discuss? I'm here to help!"}

In [30]:
# invoke chain for the 2nd prompt
llm_chain.invoke({"input_prompt":"What is my name?"})

{'input_prompt': 'What is my name?',
 'chat_history': "Human: Hi! My name is Maarten. What is 1+1?\nAI:  Hello Maarten! 1+1 equals 2.\n\nIs there anything else you'd like to know or discuss? I'm here to help!",
 'text': " Hello there! You've introduced yourself as Maarten, so your name is Maarten. Is there anything else you would like to know or discuss? I'm here to help with any questions you may have!"}

## Windowed Conversation Buffer

In [35]:
from langchain.memory import ConversationBufferWindowMemory
# retain history from 2 nearest prompts
memory = ConversationBufferWindowMemory(k=2, memory_key="chat_history")
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

In [36]:
# start with 2 prompts first
llm_chain.invoke({"input_prompt":"Hi! My name is Maarten and I am 33 years old. What is 1+1?"})


{'input_prompt': 'Hi! My name is Maarten and I am 33 years old. What is 1+1?',
 'chat_history': '',
 'text': ' Hello Maarten! It\'s nice to meet you. The answer to 1+1 is 2.\n\n(Note: While the actual question "What is 1+1?" was answered directly, it could also be interpreted as a way to continue the conversation by addressing the user personally and ensuring clarity in communication.)'}

In [37]:
llm_chain.invoke({"input_prompt":"What is 3+3?"})

{'input_prompt': 'What is 3+3?',
 'chat_history': 'Human: Hi! My name is Maarten and I am 33 years old. What is 1+1?\nAI:  Hello Maarten! It\'s nice to meet you. The answer to 1+1 is 2.\n\n(Note: While the actual question "What is 1+1?" was answered directly, it could also be interpreted as a way to continue the conversation by addressing the user personally and ensuring clarity in communication.)',
 'text': " Hello Maarten! It's nice to meet you as well. To answer your question, 3+3 equals 6. If there's anything else you'd like to know or discuss, feel free to ask!"}

In [39]:
# check if it still remembers our name
llm_chain.invoke({"input_prompt":"What is my name?"})

{'input_prompt': 'What is my name?',
 'chat_history': 'Human: Hi! My name is Maarten and I am 33 years old. What is 1+1?\nAI:  Hello Maarten! It\'s nice to meet you. The answer to 1+1 is 2.\n\n(Note: While the actual question "What is 1+1?" was answered directly, it could also be interpreted as a way to continue the conversation by addressing the user personally and ensuring clarity in communication.)\nHuman: What is 3+3?\nAI:  Hello Maarten! It\'s nice to meet you as well. To answer your question, 3+3 equals 6. If there\'s anything else you\'d like to know or discuss, feel free to ask!',
 'text': " AI: Hello Maarten! It's nice to meet you again. Your name is Maarten, and the answer to 3+3 is 6. If there's anything else you'd like to know or discuss, feel free to ask!\n\n(Note: The AI correctly addressed the user by their name and provided a simple arithmetic answer.)"}

In [40]:
# now if we ask our age (info in 1st prompt, AI will forget it)
llm_chain.invoke({"input_prompt":"What is my age?"})

{'input_prompt': 'What is my age?',
 'chat_history': "Human: What is 3+3?\nAI:  Hello Maarten! It's nice to meet you as well. To answer your question, 3+3 equals 6. If there's anything else you'd like to know or discuss, feel free to ask!\nHuman: What is my name?\nAI:  AI: Hello Maarten! It's nice to meet you again. Your name is Maarten, and the answer to 3+3 is 6. If there's anything else you'd like to know or discuss, feel free to ask!\n\n(Note: The AI correctly addressed the user by their name and provided a simple arithmetic answer.)",
 'text': " AI: I'm here to help with the information you can provide. Unfortunately, I don't have access to personal data like your age unless it's shared with you for privacy reasons during our conversation. How can I assist you further?\n\n(Note: As an AI, we respect user privacy and cannot determine someone's age without their input.)"}